# Config agent

Chat to fill in config variables in State

In [1]:
from typing import Optional, List, Union, TypedDict, Annotated

from datetime import date, datetime
from pydantic import BaseModel, Field

from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.runnables.config import RunnableConfig
from langchain_core.tools import tool

from langchain_google_vertexai import ChatVertexAI

from langgraph.graph import END, START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.checkpoint.sqlite import SqliteSaver

from promptgit import Prompt

In [2]:
class ConfigState(BaseModel):
    name: str = None
    street: str = None
    city: str = None
    messages: Annotated[list, add_messages] = []

In [3]:
@tool
def update_name(name: str, config: RunnableConfig):
    """ 
    Update name
    """
    return {'name': name}

@tool
def update_adress(street: str, city: str, config: RunnableConfig):
    """
    Update adress. Need both street and city
    """
    return {'street': street, 'city': city}

In [11]:
def initialize(state: ConfigState, config: RunnableConfig):
    if state.messages == []:
        return {'messages': [
            SystemMessage('You are setting user in the application. Your aim is to fill all the data in short conversation. Please provide name and adress (city and street)'), 
            HumanMessage('Hi'),
#            AIMessage('Please provide name and adress (city and street)')
        ]}
    else:
        return

In [12]:
def chatbot(state: ConfigState, config: RunnableConfig):
    return {'messages':[llm.invoke(state.messages)]}

In [13]:
# def tool_condition():
#     pass
from langgraph.prebuilt import tools_condition

In [14]:
llm = ChatVertexAI(model='gemini-1.5-flash-002').bind_tools([update_name, update_adress])

In [15]:
llm.invoke([
    HumanMessage('Hi'), 
    AIMessage('Let`s set up your account. What is your name and adress (street and city)'), 
    HumanMessage('Boris, Colonial Ave, Pelham')
    # HumanMessage('OK, I`m Boris'), 
    # AIMessage('Thank you, how about adress? No need to update_name'), 
    # HumanMessage('I live in Pelham'),
    # AIMessage('What is the street name?'),
    # HumanMessage('Colonial abenue')
])

This model can reply with multiple function calls in one response. Please don't rely on `additional_kwargs.function_call` as only the last one will be saved.Use `tool_calls` instead.


AIMessage(content='', additional_kwargs={'function_call': {'name': 'update_adress', 'arguments': '{"street": "Colonial Ave", "city": "Pelham"}'}}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 48, 'candidates_token_count': 14, 'total_token_count': 62, 'cached_content_token_count': 0}, 'finish_reason': 'STOP'}, id='run-1416a654-de49-4c28-86bb-e2838c3a0f92-0', tool_calls=[{'name': 'update_name', 'args': {'name': 'Boris'}, 'id': '5b8f8c60-e962-4ecc-a726-d081fdbcf8aa', 'type': 'tool_call'}, {'name': 'update_adress', 'args': {'street': 'Colonial Ave', 'city': 'Pelham'}, 'id': 'bf101d9d-3c92-47a1-8db7-00625931f752', 'type': 'tool_call'}], usage_metadata={'input_tokens': 48, 'output_tokens': 14, 'total_tokens': 62})

In [16]:
graph_builder = StateGraph(ConfigState)

graph_builder.add_node("initialize", initialize)
graph_builder.add_node("chatbot", chatbot)

graph_builder.add_edge(START, "initialize")
graph_builder.add_edge("initialize", "chatbot")
graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile()

In [17]:
graph.invoke({'messages':[]})

{'messages': [SystemMessage(content='You are setting user in the application. Your aim is to fill all the data in short conversation. Please provide name and adress (city and street)', additional_kwargs={}, response_metadata={}, id='c98fb471-0e83-45c2-9fb9-577bcd1e8c41'),
  HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}, id='885ca364-5073-41b0-8c96-858656131327'),
  AIMessage(content='Hello!  What is your name?\n', additional_kwargs={}, response_metadata={'is_blocked': False, 'safety_ratings': [], 'usage_metadata': {'prompt_token_count': 53, 'candidates_token_count': 9, 'total_token_count': 62, 'cached_content_token_count': 0}, 'finish_reason': 'STOP'}, id='run-eb5b15ce-db7e-4c23-b206-b4f31a880ba4-0', usage_metadata={'input_tokens': 53, 'output_tokens': 9, 'total_tokens': 62})]}